# Imports

##### General imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")

In [2]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import copy

##### Import from flatland environment 

In [3]:
from flatland.envs.rail_env import RailEnv
from flatland.envs.observations import *
from flatland.envs.rail_generators import complex_rail_generator,rail_from_manual_specifications_generator,random_rail_generator, RailGenerator
from flatland.envs.schedule_generators import complex_schedule_generator, random_schedule_generator, ScheduleGenerator
from flatland.utils.rendertools import RenderTool

##### Import from our framework

In [4]:
from src.graph import NetworkGraph
from src.flows import TimeNetwork
from src.flows import MCFlow

# Test of time expanded network

##### Create a flatland network

In [5]:
number_agents = 10

size_side = 7
env = RailEnv(width=size_side,
              height=size_side,
              rail_generator=complex_rail_generator(nr_start_goal=20, nr_extra=1, 
                                                    min_dist=6, max_dist=99999, 
                                                    seed = np.random.randint(0,2000)),
              schedule_generator=complex_schedule_generator(),
              number_of_agents=number_agents,
              obs_builder_object=GlobalObsForRailEnv())

env.reset()

env_renderer = RenderTool(env)
env_renderer.render_env(show=True, show_predictions=False, show_observations=False)


matrix_rail = np.array(env.rail.grid.tolist())
flatlandNetwork = NetworkGraph(matrix_rail,[(0,1)],[(1,0)])

In [6]:
sources = []
sinks = []
for agent in env.agents:
    sources.append(agent.initial_position)
    sinks.append(agent.target)

##### create a time expanded network

In [7]:
import time

In [8]:
start = time.time()
TestNetworkTime = TimeNetwork(flatlandNetwork, depth=50)
stop = time.time()
print(f'time taken to build the graph: {stop-start}')

time taken to build the graph: 0.15590929985046387


In [9]:
TestNetworkTime.connect_sources_and_sink(sources,sinks)

##### Test LP Formulation

test a simple graph

In [10]:
mcflow = MCFlow(TestNetworkTime.graph,len(sources),TestNetworkTime.get_topology_network())

Academic license - for non-commercial use only


In [11]:
mcflow.m.write("temp/model.lp")

In [12]:
mcflow.solve()

Optimize a model with 37484 rows, 47216 columns and 199248 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 31125 rows and 40079 columns
Presolve time: 0.10s
Presolved: 6359 rows, 7137 columns, 28618 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.864e+04
 Factor NZ  : 3.545e+05 (roughly 8 MBytes of memory)
 Factor Ops : 4.869e+07 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.31032500e+04 -1.24378357e+03  2.89e+02 5.21e-01  7.51e+00     0s

Barrier performed 0 iterations in 0.23 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 1473 iterations and 0.24 second

In [13]:
paths = mcflow.get_paths_solution()

In [14]:
lengths = []
pathsToAllongate = copy.deepcopy(mcflow.solution)
for agent,path in pathsToAllongate.items():
    lengths.append(len(path))
    
maxLength = max(lengths)
for agent,path in pathsToAllongate.items():
    for i in range(maxLength-len(path)):
        path.append(None)
    pathsToAllongate[agent] = path

In [15]:
dfPaths = pd.DataFrame(pathsToAllongate)
dfPaths

,0,1,2,3
0,"(5, 5)","(5, 0)","(3, 6)","(0, 2)"
1,None,None,"(3, 5)","(1, 2)"
2,None,None,None,"(2, 2)"
3,None,None,None,"(3, 2)"


In [16]:
mcflow.show_results()


Optimal flows for 0:
(5, 4)_E_in_t1 -> (5, 4)_N_out_t2: 0.833333
(5, 5)_W_out_t0 -> (5, 5)_W_out_t1: 0.166667
(5, 5)_W_out_t0 -> (5, 4)_E_in_t1: 0.833333
(5, 5)_W_out_t1 -> (5, 5)_W_out_t2: 0.166667
(5, 4)_N_out_t2 -> (4, 4)_S_in_t3: 0.833333
(5, 5)_W_out_t2 -> (5, 4)_E_in_t3: 0.166667
(4, 4)_S_in_t3 -> (4, 4)_W_out_t4: 0.833333
(5, 4)_E_in_t3 -> (5, 4)_N_out_t4: 0.166667
(4, 4)_W_out_t4 -> (4, 3)_E_in_t5: 0.833333
(5, 4)_N_out_t4 -> (5, 4)_N_out_t5: 0.166667
(4, 3)_E_in_t5 -> (4, 3)_E_in_t6: 0.166667
(4, 3)_E_in_t5 -> (4, 3)_N_out_t6: 0.666667
(5, 4)_N_out_t5 -> (4, 4)_S_in_t6: 0.166667
(4, 3)_N_out_t6 -> (3, 3)_S_in_t7: 0.666667
(4, 3)_E_in_t6 -> (4, 3)_N_out_t7: 0.166667
(4, 4)_S_in_t6 -> (4, 4)_S_in_t7: 0.166667
(3, 3)_S_in_t7 -> (3, 3)_W_out_t8: 0.666667
(4, 3)_N_out_t7 -> (3, 3)_S_in_t8: 0.166667
(4, 4)_S_in_t7 -> (4, 4)_W_out_t8: 0.166667
(3, 3)_W_out_t8 -> (3, 2)_E_in_t9: 0.666667
(3, 3)_S_in_t8 -> (3, 3)_W_out_t9: 0.166667
(4, 4)_W_out_t8 -> (4, 3)_E_in_t9: 0.166667
(3, 2)_E_